In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from unityagents import UnityEnvironment
import numpy as np
import matplotlib.pyplot as plt
from models import ACNetwork

torch.manual_seed(0)
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device ="cpu"

## Create an actor-critic model
uses 33 inputs, 4 outputs and and 64 units per hidden layer

In [2]:
# Create model
network = ACNetwork(33,4,64).double().to(device)

network.load_state_dict(torch.load("savedata/ppo_trained.save"))
network.eval();

## Load Unity environment

In [3]:
env = UnityEnvironment(file_name='Reacher_Windows_x86_64/Reacher')

# get env info
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
env_info = env.reset(train_mode=False)[brain_name] 
num_agents = len(env_info.agents)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


## Observe a PPO agent

In [4]:

tmax = 2000
max_episodes = 1

# list to contain mean scores
history = []

for episode in range(max_episodes):
    
    env_info = env.reset(train_mode=False)[brain_name] 
    states = env_info.vector_observations
    scores = 0
    episode_done = False
    

    for i in range(tmax):
        states = torch.tensor(states).to(device)
        actions= network.act_deterministic(states).detach()
        env_info = env.step(actions.cpu().clamp(-1,1).numpy())[brain_name]
        new_states = env_info.vector_observations
        rewards = env_info.rewards
        dones = env_info.local_done
        scores += np.mean(rewards)

        if np.any(dones):
            break            
        states = new_states       

print("Average reward over all 20 agents: %.3f" % scores)


Average reward over all 20 agents: 33.092


## Close environment

In [5]:
env.close()